In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle
import pandas as pd
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Data import & preprocessing 

In [3]:
import pickle
with open('../../video/lolData/exp_data/gamedata.pickle',"rb") as dt:
    data = pickle.load(dt)

In [4]:
for game in data.keys():
    data[game]['Time'] = list(map(lambda x:int(x.split('-')[0])*3600+int(x.split('-')[1])*60+int(x.split('-')[2]),data[game]['Time']))

In [5]:
for game in data.keys():
    init = data[game]['Time'][0]
    data[game]['Time'] = list(map(lambda x:x-init,data[game]['Time']))

In [6]:
for i in data:
    index = 0
    time = 0
    datas = pd.DataFrame(columns = data[i].columns)
    while(time<data[i].iloc[-1]['Time']):
        temp = data[i].iloc[[index],:]
        if int(temp['Time']) == time:
            datas = pd.concat([datas,temp],ignore_index=True)
            index+=1
        else:
            temp['Time'] = time
            datas = pd.concat([datas,temp],ignore_index=True)
        time+=1
    data[i] = datas

/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
for i in data:
    data[i] = data[i].drop(['Time','Timestamp'],axis='columns')

In [8]:
sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654','102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630','102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']

In [9]:
temp=[]
for i in data:
    if i not in sample:
        temp.append(i)
for i in temp:
    data.pop(i)

In [10]:
#변화율만 계산
for i in data:
    temp = pd.concat([data[i][data[i].index<1],data[i].drop(len(data[i])-1)],ignore_index=True)
    data[i] -= temp

In [11]:
#정규화
from sklearn.preprocessing import MinMaxScaler
min_max_scaler=MinMaxScaler()
data_scaled={}
for i in data:
    min_max_scaler.fit(data[i])
    output = min_max_scaler.transform(data[i])
    data_scaled[i] = pd.DataFrame(output,columns=data[i].columns)

# shift window, highlight 25%, dataset 분리

In [12]:
train_i = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
val_i = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
test_i = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']

In [13]:
# label load
with open('../../multimodal/label/label.pickle','rb') as f:
    label=pickle.load(f)

In [14]:
# 데이터 자르기 
for i in data_scaled:
    data_scaled[i] = data_scaled[i][:len(label[i])]

In [25]:
'''
with open('./video_raw_feature.pickle','wb') as f:
    pickle.dump(data_scaled,f)
'''

In [4]:
with open('./video_raw_feature.pickle','rb') as f:
    data_scaled = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './video_raw_feature.pickle'

In [15]:
np.array(data_scaled['102844412722519367']).shape

(2069, 10)

In [16]:
#25% 추출
label_scaled={}
for i in label:
    index=[]
    temp=[]
    for j,v in enumerate(label[i]):
        if v==1:
            temp.append(j)
        if len(temp) !=0 and (v==0 or j == len(label[i])-1):
            temp=temp[-int(len(temp)/4):]
            index+=temp
            temp=[]
    result=[]
    for j in range(len(label[i])):
        if j in index:
            result.append(1)
        else:
            result.append(0)
    label_scaled[i] = np.array(result)

In [17]:
#sliding window
data_shifted={}
for i in data_scaled:
    temp = []
    for j in range(len(data_scaled[i])):
        if j == len(label_scaled[i]):
            break
        if j+7<=len(data_scaled[i]):
            temp.append(data_scaled[i][j:j+7])
        else:
            temp.append(pd.concat([data_scaled[i][j:len(data_scaled[i])],data_scaled[i][0:j+7-len(data_scaled[i])]],ignore_index=True))
    data_shifted[i] = temp

In [18]:

with open('video_label.pickle','wb') as f:
    pickle.dump(label_scaled,f)


In [19]:

with open('video_feature.pickle','wb') as f:
    pickle.dump(data_shifted,f)


In [20]:
with open('video_feature.pickle','rb')as f:
    data_shifted = pickle.load(f)

In [21]:
with open('video_label.pickle','rb') as f:
    label_scaled = pickle.load(f)

In [22]:
#dataset 나누기
data_train=[]
label_train=np.array([])
data_val=[]
label_val=np.array([])
data_test=[]
label_test=np.array([])

for i in data_shifted:
    if i in train_i:
        temp=[]
        for j in data_shifted[i]:
            temp.append(np.array(j))
        data_train+=temp
        
        label_train = np.concatenate((label_train,label_scaled[i]),axis=None)
    elif i in val_i:
        temp=[]
        for j in data_shifted[i]:
            temp.append(np.array(j))
        data_val+=temp
        
        label_val = np.concatenate((label_val,label_scaled[i]),axis=None)
    elif i in test_i:
        temp=[]
        for j in data_shifted[i]:
            temp.append(np.array(j))
        data_test+=temp
        
        label_test = np.concatenate((label_test,label_scaled[i]),axis=None)
        
data_train = np.array(data_train)
data_val = np.array(data_val)
data_test = np.array(data_test)

# LSTM

In [23]:
weight_dir='./video_weights/'

In [24]:
# 전체 데이터에 대해서 score
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [25]:
class LSTM(nn.Module):
    def __init__(self, input=10):
        super(LSTM, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, 128, 3, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(128, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.cuda()
        h0 = ( Variable(torch.zeros(3, text.size(0), 128)).cuda(),  Variable(torch.zeros(3, text.size(0), 128)).cuda())
        
        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred,lang_feature


In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
model = LSTM().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)

In [28]:
class Dataset(torch.utils.data.Dataset):

    # Initialize your data, download, etc.
    def __init__(self,data,label):
        self.x_data = data
        self.y_data = label

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return len(self.y_data)

In [29]:
dataset = Dataset(data_train,label_train)
train_loader = DataLoader(dataset=dataset,batch_size=32)
dataset = Dataset(data_val,label_val)
val_loader = DataLoader(dataset=dataset,batch_size=32)
dataset = Dataset(data_test,label_test)
test_loader = DataLoader(dataset=dataset,batch_size=32)

In [30]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [34]:
best_losses=1000000
for epoch in range(60):
    lr = adjust_learning_rate(optimizer, epoch)
    train_loss=0
    print(epoch)
    model.train()
    for i, (text,labels) in enumerate(train_loader):
        inputs=torch.tensor(text).float()
        inputs = Variable(inputs).cuda()
        labels = torch.tensor(labels)
        labels = Variable(labels).cuda()
        optimizer.zero_grad()
        output,_=model(inputs)

        loss=criterion(output,labels.long())
        train_loss+=loss
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.) #rnn 계열 gradient 장치
        loss.backward()
        optimizer.step()


    #validation
    model.eval()
    val_loss=0
    acc=0
    gt_sum=0
    val_result ={}
    with open(weight_dir+'train_result','a') as f:
        with torch.no_grad():
            for it, (text,labels) in enumerate(val_loader):
                inputs=torch.tensor(text).float()
                inputs = Variable(inputs).cuda()
                labels = torch.tensor(labels)
                labels = Variable(labels).cuda()
                output,_=model(inputs)
                loss=criterion(output,labels.long())
                val_loss+=loss
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
                acc=acc+TP+TN
                gt_sum+=len(output)
            val_acc=acc/gt_sum
            print("epoch {} train_loss : {} , val_loss : {}, val_acc : {}".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
            f.write("epoch {} train_loss : {} , val_loss : {}, val_acc : {}\n".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
            if best_losses>val_loss:
                best_losses=val_loss
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train_best"))#epoch 변화 확인위해
                torch.save(model.state_dict(),'{}'.format(weight_dir+"train_best"))#train best 덮어써짐

                f.write("epoch {} saved\n".format(epoch))
            else:
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))


0


/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To 

epoch 0 train_loss : 0.17973992228507996 , val_loss : 0.16942070424556732, val_acc : 0.959872683869501
1
epoch 1 train_loss : 0.17968693375587463 , val_loss : 0.16941627860069275, val_acc : 0.959872683869501
2
epoch 2 train_loss : 0.17964766919612885 , val_loss : 0.16941162943840027, val_acc : 0.959872683869501
3
epoch 3 train_loss : 0.1796165108680725 , val_loss : 0.16940844058990479, val_acc : 0.959872683869501
4
epoch 4 train_loss : 0.1795918345451355 , val_loss : 0.1694047600030899, val_acc : 0.959872683869501
5
epoch 5 train_loss : 0.17957188189029694 , val_loss : 0.1694021224975586, val_acc : 0.959872683869501
6
epoch 6 train_loss : 0.17955587804317474 , val_loss : 0.16940176486968994, val_acc : 0.959872683869501
7
epoch 7 train_loss : 0.17954225838184357 , val_loss : 0.1694008708000183, val_acc : 0.959872683869501
8
epoch 8 train_loss : 0.17953065037727356 , val_loss : 0.16939863562583923, val_acc : 0.959872683869501
9
epoch 9 train_loss : 0.1795206218957901 , val_loss : 0.16939

# test

In [35]:
#test

dataset=weight_dir+'train_best' #best weight
checkpoint=torch.load(dataset,map_location='cuda:0')#model weight load
model.load_state_dict(checkpoint)
model.cuda()

#test exp
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (text,labels) in enumerate(test_loader):
        inputs= torch.tensor(text).float()
        inputs = inputs.cuda()
        labels = labels.cuda()
        output,_=model(inputs)

        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 23 9 tensor(0) tensor(9)
0 0 29 3 tensor(0) tensor(3)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 30 2 tensor(0) tensor(2)
0 0 32 0 tensor(0) tensor(0)
0 0 29 3 tensor(0) tensor(3)
0 0 31 1 tensor(0) tensor(1)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 31 1 tensor(0) tensor(1)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tenso

# feature 추출

In [44]:
save_dir = './' #feature 어디에 저장할지
dataset=weight_dir+'train_best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.cuda()
model.eval()

result={}
with torch.no_grad():
    for it, (text,labels) in enumerate(test_loader):
        inputs= torch.tensor(text).float()
        inputs = inputs.cuda()
        labels = labels.cuda()
        _,output_f=model(inputs) #feature 만 추출
        print(labels)
        if game_id[0] not in result.keys():
            result[game_id[0]]=[(output_f[0]).tolist()]
            
        else:
            result[game_id[0]]+=[(output_f[0]).tolist()]

#with open(weight_dir+save_dir,'video_final_feature') as f:
#    json.dump(result,f)     

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.float64)


/home/gmpark/anaconda3/envs/mm/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


NameError: name 'game_id' is not defined